In [1]:
import numpy as np
import pandas as pd
import time
import memory_profiler
%load_ext memory_profiler

import NaiveDE
import SpatialDE


def get_coords(index):
    coords = pd.DataFrame(index=index)
    coords['x'] = index.str.split('x').str.get(0).map(float)
    coords['y'] = index.str.split('x').str.get(1).map(float)
    return coords


In [2]:
nc=[]
tt =[]
import time
n_cell=[100,250,500,1000,1500,5000,10000]

for i in n_cell:
    ff='Data/10k_cells_100genes_counts_0.csv'
    df_0 = pd.read_csv(ff,sep=',', index_col = 0, iterator=True)

    df=df_0.get_chunk(i)
    df = df.T[df.sum(0) >= 3].T   
   
    sample_info = get_coords(df.index)
    sample_info['total_counts'] = df.sum(1)
    sample_info = sample_info.query('total_counts > 10')  # Remove empty features
    

    df = df.loc[sample_info.index]     # Align count matrix with metadata table
    X = sample_info[['x', 'y']]

# Convert data to log-scale, and account for depth
    dfm = NaiveDE.stabilize(df.T).T
    res = NaiveDE.regress_out(sample_info, dfm.T, 'np.log(total_counts)').T

# Add total_count as pseudogene for reference
    res['log_total_count'] = np.log(sample_info['total_counts'])

# Perform Spatial DE test with default settings
    start=time.time()
    %memit results = SpatialDE.run(X, res)
    finish=time.time()
    t=finish-start                # 15706 
    
    nc.append(int(i))    
    tt.append(t)


In [4]:
tt

[3.78774356842041,
 4.402992248535156,
 5.221548795700073,
 7.5988616943359375,
 15.450058460235596,
 256.7932643890381,
 1967.3115754127502]

In [5]:
memory=[109.96,118.01 ,143.00,234.04 , 386.53, 2775.26,10872.72 ]

In [6]:
time=pd.DataFrame({'n.cells':nc,'n.genes':100,'time_s':tt,'Memory_MiB':memory})

In [7]:
time

,n.cells,n.genes,time_s,Memory_MiB
0,100,100,3.787744,109.96
1,250,100,4.402992,118.01
2,500,100,5.221549,143.00
3,1000,100,7.598862,234.04
4,1500,100,15.450058,386.53
5,5000,100,256.793264,2775.26
6,10000,100,1967.311575,10872.72
